In [48]:
import pandas as pd
import fiona
from shapely.geometry import shape, mapping
import itertools
from collections import OrderedDict

In [49]:
# load the shapefile as an iterable
polys = fiona.open("capas/ine_deptos_generalizada.gpkg")

In [50]:
def shared_boundary(file):
    polys = fiona.open(file)
    shared = []

    for pol1, pol2 in  itertools.combinations(polys, 2):
        geompol1 = shape(pol1['geometry'])
        geompol2 = shape(pol2['geometry'])
        if geompol1.touches(geompol2):
            length = shape(mapping(geompol1.intersection(geompol2))).length
            shared.append([tuple(pol1['properties'].items())[0], tuple(pol2['properties'].items())[0], int(length)])
    
    shared = list(zip([x[0][1] for x in shared],
                  [x[1][1] for x in shared],
                  [x[2] for x in shared]                  
                 ))
    
    df = pd.DataFrame(shared, columns=['poly1', 'poly2', 'length']).sort_values(by=['poly1', 'poly2'])
    df = df.loc[df.length > 0].reset_index(drop=True)

    return df

In [51]:
file="capas/ine_deptos_generalizada.gpkg"

df = shared_boundary(file)

<ipython-input-50-e1da82376c14>:5: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for pol1, pol2 in  itertools.combinations(polys, 2):


In [52]:
df.rename({'poly1': 'dep1', 'poly2': 'dep2', 'length': 'largo_limite'}, inplace=True, axis=1)
print(df.shape)
df.head()

(39, 3)


,dep1,dep2,largo_limite
0,3,1,60107
1,3,8,94070
2,3,9,51167
3,3,10,24113
4,3,16,48749


In [53]:
def codificar(df, var1, var2):
    df['cod'] = (df[var1].astype(str) + df[var2].astype(str).str.zfill(2)).astype(int)
    
    return df

In [54]:
# codifica
df1 = codificar(df, 'dep1', 'dep2')
df2 = codificar(df, 'dep2', 'dep1')

In [55]:
deptos_vecinos = df1.loc[:,['cod','largo_limite']].append(df2.loc[:,['cod','largo_limite']]).drop_duplicates('cod').sort_values('cod')

In [56]:
print(deptos_vecinos.shape)
deptos_vecinos.head()

(39, 2)


,cod,largo_limite
0,103,60107
31,116,5400
27,215,197272
24,413,64044
32,516,67607


In [57]:
deptos_vecinos.cod.unique().shape

(39,)